<a href="https://colab.research.google.com/github/Raziel619/notebooks/blob/branch-0.8/colaboratory/cuml/sgd_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stochastic gradient descent (SGD) 
SGD is an  incremental gradient descent algorithm which modifies its weights, in an effort to reach a local minimum. The cuML implementation takes only numpy arrays and cuDF datasets as inputs. In order to convert your dataset into a cuDF dataframe format please refer the documentation on https://rapidsai.github.io/projects/cudf/en/latest/. The SGD algorithm implemented in cuML can accept the following parameters:
1. loss : 'hinge', 'log', 'squared_loss' (default = 'squared_loss')
2. penalty: 'none', 'l1', 'l2', 'elasticnet' (default = 'none')
3. alpha: float (default = 0.0001)
4. fit_intercept : boolean (default = True)
5. epochs : int (default = 1000)
6. tol : float (default = 1e-3)
7. shuffle : boolean (default = True)
8. eta0 : float (default = 0.0)
9. power_t : float (default = 0.5)
10. learning_rate : 'optimal', 'constant', 'invscaling', 'adaptive' (default = 'constant')
11. n_iter_no_change : int (default = 5)

For additional information on the SGD model please refer to the documentation on https://rapidsai.github.io/projects/cuml/en/latest/index.html


In [1]:
#ensure that the runtime is set to GPU
#use !nvidia-smi to make sure a Tesla T4 is being used 
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c conda-forge \
  -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
  cudf cuml

# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
  shutil.copy('/usr/local/lib/'+fn, os.getcwd())

--2019-06-04 19:52:10--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   148MB/s    in 0.4s    

2019-06-04 19:52:11 (148 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-

In [0]:
import numpy as np
import pandas as pd
import cudf
import os
from cuml.solvers import SGD as cumlSGD
from sklearn.linear_model import SGDRegressor
# Select a particular GPU to run the notebook  
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Helper Functions

In [0]:
# check if the mortgage dataset is present and then extract the data from it, else just create a random dataset for sgd 
import gzip
# change the path of the mortgage dataset if you have saved it in a different directory
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz'):
    if os.path.exists(cached):
        print('use mortgage data')

        with gzip.open(cached) as f:
            X = np.load(f)
        # the 4th column is 'adj_remaining_months_to_maturity'
        # used as the label
        X = X[:,[i for i in range(X.shape[1]) if i!=4]]
        y = X[:,4:5]
        rindices = np.random.randint(0,X.shape[0]-1,nrows)
        X = X[rindices,:ncols]
        y = y[rindices]

    else:
        # create a random dataset
        print('use random data')
        X = np.random.rand(nrows,ncols)
        y = np.random.randint(0,10,size=(nrows,1))
    train_rows = int(nrows*0.8)
    df_X_train = pd.DataFrame({'fea%d'%i:X[0:train_rows,i] for i in range(X.shape[1])})
    df_X_test = pd.DataFrame({'fea%d'%i:X[train_rows:,i] for i in range(X.shape[1])})
    df_y_train = pd.DataFrame({'fea%d'%i:y[0:train_rows,i] for i in range(y.shape[1])})
    df_y_test = pd.DataFrame({'fea%d'%i:y[train_rows:,i] for i in range(y.shape[1])})
    return df_X_train, df_X_test, df_y_train, df_y_test


In [0]:
# this function checks if the results obtained from two different methods (sklearn and cuml) are the same
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=2e-3,with_sign=True):
    a = to_nparray(a).ravel()
    b = to_nparray(b).ravel()
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

# the function converts a variable from ndarray or dataframe format to numpy array
def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x


# Run tests

In [5]:
%%time
# nrows = number of samples
# ncols = number of features of each sample
nrows = 2**20
ncols = 399

# dataset is split into a ratio of 80:20, 
# 80% is used as the training data and the remaining 20% is used as the test data
X_train, X_test, y_train, y_test = load_data(nrows,ncols)
y_train_ser = y_train['fea0']
print('training data',X_train.shape)
print('training label',y_train.shape)
print('testing data',X_test.shape)
print('testing label',y_test.shape)

use random data
training data (838860, 399)
training label (838860, 1)
testing data (209716, 399)
testing label (209716, 1)
CPU times: user 16.1 s, sys: 3.61 s, total: 19.7 s
Wall time: 19.8 s


Here we set the parameters usedby both libraries. You can change the number of iterations used by changing the `iterations` variable.  Please note that making this too high can cause the functions to take a long time to complete.

In [0]:
#set parameters 
learning_rate = 'adaptive'
datatype = np.float32
penalty = 'elasticnet'
loss = 'squared_loss'
iterations = 10 

The `max_iter` parameter controls the maxixmum number of iterations the model can run for but it doesn’t guarantee that the model will definitely run for all those epochs, therefore the sklearn might run for less number of epochs than the cuML model

In [7]:
%%time
# use the sklearn SGD Regressor model to fit the dataset 
sk_sgd = SGDRegressor(learning_rate=learning_rate, eta0=0.07,
                       max_iter=iterations, tol=0.0, fit_intercept=True,
                       penalty=penalty, loss=loss)
sk_sgd.fit(X_train, y_train_ser)


CPU times: user 44.8 s, sys: 52.6 ms, total: 44.8 s
Wall time: 44.9 s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:1185: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [8]:
%%time
# test the model by predicting its results for the unseen test set
y_sk = sk_sgd.predict(X_test)

# calculate the Mean Squared Error for the model's predictions
error_sk = mean_squared_error(y_test,y_sk)


CPU times: user 195 ms, sys: 3.93 ms, total: 199 ms
Wall time: 149 ms


In [9]:
%%time
# convert the pandas dataframe to cuDF dataframe and series
X_cudf = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)
y_cudf = cudf.Series(y_train_ser)

CPU times: user 3.33 s, sys: 1.7 s, total: 5.03 s
Wall time: 5.29 s


In [10]:
%%time
# fit the training data on cuML's implementation of SGD
cu_sgd = cumlSGD(learning_rate=learning_rate, eta0=0.07, epochs=iterations, #epochs == n_iter
                 batch_size=512,
                 tol=0.0, penalty=penalty, loss=loss)
cu_sgd.fit(X_cudf, y_cudf)


CPU times: user 4 s, sys: 1.35 s, total: 5.35 s
Wall time: 5.35 s


In [11]:
%%time
# test the model by predicting its values for the test set
y_pred = cu_sgd.predict(X_cudf_test)
y_pred = to_nparray(y_pred).ravel()
# calculate the Mean Squared Error for the model's predictions
error_cu = mean_squared_error(y_test,y_pred)

CPU times: user 110 ms, sys: 22.8 ms, total: 132 ms
Wall time: 136 ms


In [12]:
# print the MSE of the sklearn and cuML models to compare them
print("SKL MSE(y):")
print(error_sk)
print("CUML MSE(y):")
print(error_cu)

SKL MSE(y):
9.447812571845638
CUML MSE(y):
28.492871311678652
